In [1]:
from flask import Flask, render_template, request
from googleapiclient.discovery import build
from transformers import pipeline
import torch

nlp = pipeline("fill-mask", model="cl-tohoku/bert-base-japanese-char")
def insert_char_to_sentence(i, char, sentence): # sentenceのi文字目にcharを挿入する
    l = list(sentence)
    l.insert(i, char)
    text = "".join(l)
    return text

video_id='0FwuF1f2jOQ'
from youtube_transcript_api import YouTubeTranscriptApi

captions=YouTubeTranscriptApi.get_transcript(video_id,languages=['ja'])
# captionをひとまとまりにする
caption=''
for i in range(len(captions)):
    caption+=captions[i]['text']
# 日本語テキストにおいて然るべきところに句点を入れる
thresh = 0.5 # このスコア以上の場合、句読点を挿入する
i = 0
punctuations = ["、", "。", "?"]
chars_after_mask = 100
corrected_sentence = caption
while i < len(corrected_sentence):
    i += 1
    if corrected_sentence[i-1] in punctuations: continue # 句読点が連続してくることはない
    masked_text = insert_char_to_sentence(i, nlp.tokenizer.mask_token, corrected_sentence)
    
    pre_context, post_context = masked_text.split("。")[-1].split(nlp.tokenizer.mask_token)
    res = nlp(f"{pre_context}{nlp.tokenizer.mask_token}{post_context[:chars_after_mask]}")[0] # scoreが一番高い文
    if res["token_str"] not in punctuations: continue
    if res["score"] < thresh: continue

    punctuation = res["token_str"] if res["token_str"] != "?" else "。" # 今回は"？"は"。"として扱う
    corrected_sentence = insert_char_to_sentence(i, punctuation, corrected_sentence)


/Users/immisius/Desktop/coding/SUtube/sutube/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-char were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
sentence="おはようございます。今日はいい天気ですね。しかし、明日は雨が降るようです。だから、傘を持っていくべきです。それでは、また明日。"

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("tsmatz/mt5_summarize_japanese")

model = AutoModelForSeq2SeqLM.from_pretrained("tsmatz/mt5_summarize_japanese")

In [16]:
inputs = tokenizer.encode(caption, return_tensors="pt", max_length=10000, truncation=True)

outputs = model.generate(inputs, max_length=1500, min_length=500, length_penalty=2.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'米モーター天国のレインは12日、1994年の撤退までのエピソードを解説します 新型コロナウイルスの革命的なマシン「F1」で多くの栄光をつかみ出してきた名門ロータスが1992年から1994年に退退を決めたことについて解説するチャンネル ではこのほど 様々な検証解説動画を解説 し、世界中の英雄を次々と生み出した。しかしその歴史は90年代後半から90年を迎えた。そして90年代の後半からは多くの革命的マシンが次々と復活を果たすことになったのですしかしこの2年では、1995年に第1戦フランス選手権で第2戦日本選手権では第3戦ドイツ選手権に出場したチームはこれまでで最も偉大な成功を収めてきた。この1ではこれまでよりも多くの英雄な名車を生みが次々と大成功したとみられるという歴史的な出来事が実現したのですそしてこの年は1990年以降で最大規模の改定に直面している。その多くは、その素晴らしく素晴らしい歴史をあらためて世界最古く輝きを見せたその伝説の「英雄」と呼ばれている。またこの次世界大戦を繰り広げてきたこの3年の歴史を振り返るその感動を打破し歴史に残る歴史的名車を次々と再建を主導していたチームが、このチームにとって最も英雄的な復活を象徴する「W杯」を復活させると、多くの感動的な勝利を喜ぶべきだったのです その素晴らしい作品と歴史の長い歴史と伝統を創り上げたことだったという ―― 今回の動画 はBBCのラジオ番組 でこの番組では 最後にこの作品 が公開された こと の最後にはこの動画ではBBCがBBCに投稿 した 今年のW1 に続き、1990年から1995年にかけて数々の数々の名作を称賛を呼ぶことで、そしてその記録をお伝えする と とても素晴らしいことになりつつあることを覚えたこの 素晴らしい記録 をお届け しますそして最後に 大きな感動を覚えている というこのビデオ や 過去の作品や数々の作品はこれまでに数々の英雄に突入していた 世界最高峰の伝説的な大進歩を加速させるため、世界最高の歴史や伝統的な歴史が再び始まって以来、数々の素晴らしい成功へと発展した。さらに多くの成功に成功することになった のですその内容はBBC ・BBCニュース には多くの素晴らしい感動の映像をまとめた ほか、BBCは 最新1のインタビュー について 解説を さらに解説していく 。'

In [34]:
inputs = tokenizer.encode(corrected_sentence[:1000], return_tensors="pt", max_length=10000, truncation=True)

outputs = model.generate(inputs, max_length=150, min_length=100, length_penalty=1e2, num_beams=100, early_stopping=True)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'ラグビーワールドカップ(W杯)日本大会(F1)で多くの栄光をつかみ、革命的なマシンを生み出した名門ロータスの壮絶な撤退までのエピソードを解説するチャンネルが24日、モーター天国関連チャンネルで放送され、世界最大規模のコンストラクターズランキング1位(世界ランキング11位)を公表します。この番組では、100年代後半から80年代の低迷を繰り広げ、90年代前半から90年を迎えた。しかし、1990年から91年にかけては、数々の革命的マシンの復活を果たした。'

In [35]:
inputs = tokenizer.encode(caption[:1000], return_tensors="pt", max_length=10000, truncation=True)

outputs = model.generate(inputs, max_length=150, min_length=100, length_penalty=1e2, num_beams=100, early_stopping=True)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'ラグビーワールドカップ(W杯)日本大会F1で多くの栄光をつかみ革命的なマシンを多数生み出してきた名門ロータスが90年代後半から1994年の撤退までのエピソードを解説しますモーター天国関連チャンネルでは様々な検証解説動画を投稿しよろしければチャンネル登録をしておいてもらえるとうれしい声が上がっている。新型コロナウイルスのロックダウン(都市封鎖)や大規模な体制変更、再建を図った第8のフォードコス(V8)の欧州連合(EU)Cシリーズ5を搭載したコンストラクターズランキング11位を獲得したトーマス・アダノム・ホプキンス最高経営責任者(CFO)準々決勝トーナメント'

inputs = tokenizer.encode(caption[:1000], return_tensors="pt", max_length=10000, truncation=True)

outputs = model.generate(inputs, max_length=150, min_length=100, length_penalty=1e2, num_beams=100, early_stopping=True)
tokenizer.decode(outputs[0], skip_special_tokens=True)

In [19]:
inputs = tokenizer.encode(corrected_sentence[101:200], return_tensors="pt", max_length=10000, truncation=True)

outputs = model.generate(inputs, max_length=1500, min_length=500, length_penalty=2.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'世界1位のF1グランプリの決勝進出を狙った、1989年に撤退までのエピソードと、当時のマシンについて解説するチャンネルで、このチャンネルでは、2020年の退退までエピソードを解説していく動画動画を販売します。BBCのチャンネルYouTubeで公開された、新型コロナウイルスの感染拡大防止策をまとめた動画が、YouTubeに投稿された動画を公開し、動画共有サイト「フォー1」が配信される。これにより、1994年に退去までの経緯や、過去のマシンを検証する動画も含まれています。この動画ではF2グランプリは、第1次世界大戦(F1)決勝に出場した。しかし、その経緯と歴史的経緯について、1990年の撤退出までのビデオを公開する動画を提供するYouTubeチャンネルには、多くの検証解説動画を無料で提供すると、インターネット上の動画や動画配信動画を配信している。その動画はYouTubeのYouTube上で公開されれば、BBCに登場する。新型は「F 1グランプリ」の2度目の決勝戦となる。さらに、決勝戦の末に、ついに、1987年を振り返る。 新型の1決勝を発表した 。2020年、脱退の経緯を説明する動画 をお届け すると発表した。1990年以降、1991年から、1992年退脱までの映像を放送する という動画の配信 が発表 される予定だ。また、F-1グランプリ(世界2位)に続き、2015年1位を獲得したとみられることが明らかになった。2015年に、退退出まで、その後の経緯が解説 します ) のビデオ を公開した と 米動画チャンネル で発表された 動画サイト YouTube に登場した ESPN に出演した動画ランキング から、2019年の、最高優勝の最高記録を更新 した、2016年2月1回優勝を目前に発表されるのは初めて。1989年までに、2000年ぶりに公開 予定 だった 、2011年3月2日に、世界3位を公表 されたこの番組 では、「1覇賞」を受賞した2020年に引退を記念して、2001年4位だった。F3覇を正式に公表された。2016年のF5位となる、初の決勝トーナメントの勝利に成功したという。2019年7位、W1優勝に輝きを見せた、これまでのマシンの「1グランド優勝」と名付けられた、同国への挑戦に直面する、新たな挑戦を披露するビデオチャンネルを開設 し た。'